In [ ]:
#Importando Pacotes:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
import geopandas as gpd
import altair as alt
import os as os
#Configurando Altair:
alt.renderers.enable('notebook')
#Importando Módulos:
from freedom import freedom
from data import data
from data import data2
from analyze import analyze
#Importando Mais Limpeza de dados:
import clean_data as cd

In [ ]:
#1. GERNADO PRIMEIRA TRAJETÓRIA DO PIB:
list_year= [str(num) for num in range(1968,2017,3)]
for year in  list_year:
    #Dados do PIB per capita de 'year':
    gdp = analyze(data,"GDP per capita (current US$)")
    gdp2017 = pd.DataFrame(gdp[year])
    #Mapa do Mundo:
    world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
    #Eliminando a Antarctica:
    world = world[(world.pop_est>0) & (world.name!="Antarctica")]
    #Mesclando as tabelas:
    merged = world.set_index('name').join(gdp2017)
    #Definindo dimensões e variáveis para o gráfico e legenda:
    variable = year
    vmin,vmax = 0,100
    fig, ax = plt.subplots(1, figsize=(15, 8))
    #Plotando o gráfico:
    imagem=merged.plot(column=variable, cmap='Reds',linewidth=0.5, ax=ax, edgecolor='0.1') 
    #Colocando legenda:
    ax.axis('off')
    ax.set_title("GDP per capita (current US$)" + year, fontdict={'fontsize': '25', 'fontweight' : '3'})
    ax.annotate('Fonte: Banco Mundial',xy=(0.1, .08),  xycoords='figure fraction', horizontalalignment='left', verticalalignment='top', fontsize=12, color='#555555')
    sm = plt.cm.ScalarMappable(cmap='Reds', norm=plt.Normalize(vmin=vmin, vmax=vmax))
    sm._A = []
    cbar = fig.colorbar(sm)
    #Salvando cada imagem do diretório
    caminho = os.path.join('Pastadogif', year+'_gdp.png')
    grafico = imagem.get_figure()
    grafico.savefig(caminho, dpi=300)

In [ ]:
#2. ANALISE DO MERCADO DE CAPITAIS:

#2.1 Criando DataFrame resumido tomando apenas anos com grande quantidade de dados 
market = analyze(data,"Market capitalization of listed domestic companies (current US$)")[[str(num) for num in range(1987,2018)]]
#Criando variáveis para guardar resultados das concatenações no for e que transforme inteiros em strings para que possamos acessá-las:
list_year= [str(num) for num in range(1987,2018)]
top_market=pd.DataFrame()
#Interação para obter apenas os quatro países com maior mercado de capitais em cada ano, mantendo a informação desse pais em todos os anos
for i in list_year:
    market = pd.DataFrame(market[i])
    market = market[market[i]>0].sort_values(i)[-4:]
    top_market = pd.concat([top_market, market], axis=1, sort=False)
    market = analyze(data,"Market capitalization of listed domestic companies (current US$)")[[str(num) for num in range(1987,2018)]]
#Eliminando variáveis Nan.
top_market=top_market.fillna(0)
top_market=market.T[top_market.index].T
top_market

In [ ]:
#2.2 Tornando tabela mais fácil de trabalhar no altair
top_market_chart=pd.DataFrame()
for i in top_market.index:
        x=pd.DataFrame({'year':top_market.columns,'Country':[i]*len(top_market.columns),'Market Capitalization':top_market.T[i]})
        top_market_chart=pd.concat([top_market_chart,x])
top_market_chart=top_market_chart.fillna(0)
top_market_chart

In [ ]:
#2.3 Plotando gráfico
chart=alt.Chart(top_market_chart).mark_area().encode(
    x='year:T',
    y='Market Capitalization:Q',
    color='Country:N'
)
chart

In [ ]:
#3 Crindo gráfico de comparação da taxa real de juros americana e pib mundial

#3.1 Criando tabela do pib mundial que seja possível juntar com taxa real de juros americana de modo que seja fácil a utilização do altair
gdp_world = cd.clean_tb(data2[0]).T['World']
gdp_world = pd.DataFrame({'Year':gdp_world.index[3:],'Gdp_World':gdp_world[3:]})
gdp_world=gdp_world[3:-1]
gdp_world

In [ ]:
#3.2 Criando tabela da taxa real de juros americana
real_rate = analyze(data,"c-United States",['Real interest rate (%)'])
real_rate.reset_index(level=0, inplace=True)
real_rate.columns=['Year','Real_rate_us']
real_rate.index=gdp_world.index
real_rate

In [ ]:
#3.3 Plotando Gráfico
join = pd.DataFrame(gdp_world).join(real_rate['Real_rate_us'])
alt.renderers.enable('notebook')
brush = alt.selection(type='interval', encodings=['x'])

real_rate_chart = alt.Chart(join).mark_line().encode(
  x=alt.X('Year:N'),
  y=alt.Y('Real_rate_us:Q')
).properties(
  title='Taxa Real de Juros Americana'
).add_selection(
    brush
)


gdp_world_chart = alt.Chart(join).mark_line().encode(
  x=alt.X('Year:N', scale=alt.Scale(zero=False)),
  y=alt.Y('Gdp_World:Q', scale=alt.Scale(zero=False)),
).properties(
  title='PIB Mundial percapita'
).transform_filter(
  brush
)


real_rate_chart | gdp_world_chart